In [6]:
import os
from ultralytics import YOLO
import cv2
import json
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
#Read API key from file
API_KEY = os.path.join(os.getcwd(), './robotflow_api.key')
API_KEY = open(API_KEY, 'r').read().strip()

MODEL_PATH = os.path.join(os.getcwd(), './runs/detect/train8/weights/best.pt')
DIR_PATH = os.path.join(os.getcwd(), '../nerf_vision/processed/5_pts/')
SAVED_PATH = os.path.join(DIR_PATH, './masks/')
if not os.path.exists(SAVED_PATH):
    os.makedirs(SAVED_PATH)

In [8]:
from roboflow import Roboflow
rf = Roboflow(api_key=API_KEY)
project = rf.workspace().project("jackal_detection")
model = project.version(4).model


loading Roboflow workspace...
loading Roboflow project...


In [9]:
#Open transforms.json file

with open(os.path.join(DIR_PATH, 'transforms.json')) as f:
    data = json.load(f)

for i in range(len(data['frames'])):
    img_path = os.path.join(DIR_PATH,data['frames'][i]['file_path'])

    #Open image
    img = cv2.imread(img_path)
    results = model.predict(img).json()
    box = results["predictions"][0]

    #Create black and white mask
    mask = np.zeros((img.shape[0],img.shape[1]),dtype=np.uint8)
    #Set mask to 1 where bounding box is
    left_x = int(box['x']-box['width']//2)
    right_x = int(box['x']+box['width']//2)
    top_y = int(box['y']-box['height']//2)
    bottom_y = int(box['y']+box['height']//2)
    mask[top_y:bottom_y,left_x:right_x] = 1

    #Save mask as png
    SAVED_MASK_IMG_PATH = "masks/"+str(i)+".png"
    cv2.imwrite(os.path.join(DIR_PATH,SAVED_MASK_IMG_PATH), mask)
    
    data['frames'][i]['mask_path'] = SAVED_MASK_IMG_PATH

    if False:
        #Two subplot
        fig, (ax1, ax2) = plt.subplots(1, 2)
        ax1.imshow(img)
        rect = matplotlib.patches.Rectangle((box['x']-box['width']/2,box['y']-box['height']/2),box['width'],box['height'],linewidth=1,edgecolor='r',facecolor='none')
        ax1.add_patch(rect)
        ax2.imshow(mask)


In [ ]:
#Save data as transforms.json
with open(os.path.join(DIR_PATH, "transforms.json"), 'w') as outfile:
    json.dump(data, outfile,indent=2)
